In [1]:
from darts import TimeSeries
from darts.models import NBEATSModel, NHiTSModel, TransformerModel, TSMixerModel
from darts.utils.losses import *
from darts.metrics import metrics as darts_metrics
from utils import data_handling, helpers
import torch
import numpy as np
import os
import pickle
import pandas as pd
import config
import copy

# Constants
DEVICE = [1]
NUM_EPOCHS = 5
IN_LEN = 96
OUT_LEN = 96
LOSS_FN = torch.nn.MSELoss()
LAYER_WIDTH = 256
NUM_STACKS = 4
NUM_BLOCKS = 2
NUM_LAYERS = 2
COEFFS_DIM = 5
DROPOUT = 0.2
VERBOSE = False
TRAIN_EPOCHS = 15
TUNE_EPOCHS = 5
four_weeks = -24*7*4

metrics_output_path = config.CONFIG_OUTPUT_PATH["darts"] / "darts_metrics.csv"

/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
def extend_source_to_target_id_count(source, target):
    source_id_count = source["train"].shape[1]
    target_id_count = target["train"].shape[1]

    full_repeats = target_id_count // source_id_count
    remainder = target_id_count % source_id_count

    repeated_tensor = source["train"].repeat(1, full_repeats)
    remainder_tensor = source["train"][:, :remainder]
    source_train = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    
    assert target_id_count == source_train.size(1), f"Reshaping was incorrect. Target_train = {target_id_count}, source_train = {source_train.size(1)}."

    repeated_tensor = source["validation"].repeat(1, full_repeats)
    remainder_tensor = source["validation"][:, :remainder]
    source_validation = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    assert target_id_count == source_validation.size(1), f"Reshaping was incorrect. Target_val = {target_id_count}, source_val = {source_validation.size(1)}."

    return source_train, source_validation


def process_tl_data(source_data, target_data):
    # either reshape source or target dataset according to which has less IDs
    source_ids = source_data["train"].size(1)
    target_ids = target_data["test"].size(1)

    fine_tune_horizon = -24*7*4
    target_test = target_data["test"]
    target_fine_tuning = target_data["train"][fine_tune_horizon:,:]

    # remove IDs if source is bigger than target or
    # repeat IDs if target is bigger than source
    if target_ids < source_ids:
        source_train = source_data["train"][:,:target_ids]
        source_validation = source_data["validation"][:,:target_ids]
    else:
        source_train, source_validation = extend_source_to_target_id_count(source_data, target_data)

    # convert to TimeSeries dataframe
    source_train = TimeSeries.from_values(source_train)
    source_validation = TimeSeries.from_values(source_validation)
    target_test = TimeSeries.from_values(target_test)
    target_fine_tuning = TimeSeries.from_values(target_fine_tuning)
    target_train = TimeSeries.from_values(target_data["train"])
    target_validation = TimeSeries.from_values(target_data["validation"])

    tl_dataset = {
                    "source_train" : source_train,
                    "source_validation" : source_validation,
                    "target_fine_tuning" : target_fine_tuning,
                    "target_test" : target_test,
                    "target_train" : target_train,
                    "target_validation" : target_validation
                }

    return tl_dataset

In [3]:
def train_nhits(ts_train, ts_val, epochs=1):
    nhits_model = NHiTSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        activation='ReLU',
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        dropout=DROPOUT,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu",  "devices" : DEVICE},
    )
    nhits_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nhits_model

# Train NBEATS model
def train_nbeats(ts_train, ts_val, epochs=1):
    nbeats_model = NBEATSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        batch_size=32,
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        expansion_coefficient_dim=COEFFS_DIM,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        activation='ReLU',
        #optimizer_kwargs={"lr": 0.0005},
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu",  "devices" : DEVICE},
    )
    nbeats_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nbeats_model

# Train Transformer model
def train_transformer(ts_train, ts_val, epochs=1):
    transformer_model = TransformerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN,
        d_model=LAYER_WIDTH, 
        nhead=4, 
        num_encoder_layers=3, 
        num_decoder_layers=3, 
        dim_feedforward=LAYER_WIDTH, 
        dropout=DROPOUT, 
        activation='relu', 
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu",  "devices" : DEVICE},

    )
    transformer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return transformer_model

# Train TSMixer model
def train_tsmixer(ts_train, ts_val, epochs=1):
    tsmixer_model = TSMixerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN, 
        hidden_size=LAYER_WIDTH, 
        ff_size=LAYER_WIDTH, 
        num_blocks=NUM_BLOCKS, 
        activation='ReLU', 
        dropout=DROPOUT, 
        loss_fn=LOSS_FN,
        norm_type='LayerNorm', 
        use_reversible_instance_norm=True,
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu", "devices" : DEVICE},

    )
    tsmixer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return tsmixer_model

In [4]:
def evaluate(model, target_test):
    """
    Evaluates models on target test set
    Input:  -trained model
            -List of target test sets shaped according to models

    Output: Dict{MSE, MAE}
    """

    forecasting_endpoint = int(len(target_test)) - 96*2
    window = [target_test[i:i+96] for i in range(0, forecasting_endpoint, 5)]
    target = [target_test[i+96:i+96+96] for i in range(0, forecasting_endpoint, 5)]

    # predict over dataloader with slidingwindow implementation and 5 time step shifts for each input
    predictions = model.predict(n=96, series=window)

    mse = darts_metrics.mse(predictions, target)
    mae = darts_metrics.mae(predictions, target)

    mse = sum(mse) / len(predictions)
    mae = sum(mae) / len(predictions)

    return {'MSE': mse, 'MAE': mae}


def fine_tune_model(model, target_fine_tuning, epochs=1):
    """
    Fine tune models over specified epochs

    Input:  -trained models
            -fine tuning dataset
            -epochs

    Returns: fitted models
    """

    model.fit(
            target_fine_tuning,
            num_loader_workers=4,
            epochs=epochs,
            max_samples_per_ts=None,
        )
 
    return model

In [5]:
# use electricity dataset
electricity_dict = data_handling.format_electricity()

for key, value in electricity_dict.items():
			electricity_dict[key]= data_handling.df_to_tensor(value)

# normalize train and use matrics for val and test
electricity_dict["4_weeks_train"] = electricity_dict["train"][four_weeks:,:]
electricity_dict["train"], train_standardize_dict = helpers.custom_standardizer(electricity_dict["train"])
electricity_dict["validation"], _ = helpers.custom_standardizer(electricity_dict["validation"], train_standardize_dict)
electricity_dict["test"], _ = helpers.custom_standardizer(electricity_dict["test"], train_standardize_dict)
electricity_dict["4_weeks_train"], _ = helpers.custom_standardizer(electricity_dict["4_weeks_train"], train_standardize_dict)

# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
bavaria_dict["4_weeks_train"] = bavaria_dict["train"][four_weeks:,:]

# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
gp_dict["4_weeks_train"] = gp_dict["train"][four_weeks:,:]

Length train set: 209 days, 0:00:00
Length validation set: 34 days, 0:00:00
Saving train, validation and test df for faster loading


In [6]:
tl_setups = {
    "ELD_to_Bavaria" : (electricity_dict, bavaria_dict), 
    "ELD_to_GP2" : (electricity_dict, gp_dict),
    "Bavaria_to_ELD" : (bavaria_dict, electricity_dict), 
    "Bavaria_to_GP2" : (bavaria_dict, gp_dict), 
    "GP2_to_Bavaria": (gp_dict, bavaria_dict), 
    "GP2_to_ELD" : (gp_dict, electricity_dict)
     }

In [7]:
# Initialize the DataFrame
models = {
    "NHiTS": train_nhits,
    "NBEATS": train_nbeats,
    "Transformer": train_transformer,
    "TSMixer": train_tsmixer
}

try:
    results_df = pd.read_csv(metrics_output_path, index_col=[0, 1, 2])
except FileNotFoundError:
    metrics = ["MSE", "MAE"]
    learning_scenarios = ["Zero-Shot", "four_weeks_tl", "full_tl", "full_training", "four_weeks_training"]
    index = pd.MultiIndex.from_product([tl_setups.keys(), learning_scenarios, metrics], names=["Setup", "Learning_scenario", "Metric"])
    results_df = pd.DataFrame(columns=models.keys(), index=index)

# Helper functions
def update_metrics(setup_name, model_name, learning_scenario, mae, mse):
    results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
    results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse

def is_metric_filled(setup_name, model_name, learning_scenario):
    # Check if specific metrics for a model in a setup and fine-tuning scenario are NaN or not
    metrics_filled = not results_df.loc[(setup_name, learning_scenario, slice(None)), model_name].isnull().any()
    return metrics_filled

def run_model_tl(setup_name, tl_data):
    source_train = tl_data["source_train"]
    source_val = tl_data["source_validation"]
    target_fine_tuning = tl_data["target_fine_tuning"]
    target_test = tl_data["target_test"]
    target_train = tl_data["target_train"]

    for model_name, model_func in models.items():
        if is_metric_filled(setup_name, model_name, "full_tl"):
            print(f"Skipping {model_name} for {setup_name} with full fine-tuning as metrics are already filled.")
            continue
        
        # Train model with source dataset (Zero-Shot)
        model = model_func(source_train, source_val, epochs=TRAIN_EPOCHS)
        metrics = evaluate(model, target_test)

        # Create deep copy for full fine-tuning
        full_tl_model = copy.deepcopy(model)

        update_metrics(setup_name, model_name, "Zero-Shot", metrics['MAE'], metrics['MSE'])
        print(f"Metrics updated for {model_name} in {setup_name} with zero-shot fine-tuning: MAE = {metrics['MAE']}, MSE = {metrics['MSE']}")
        
        # Fine-tune on small target train set (four_weeks_tl)
        model = fine_tune_model(model, target_fine_tuning, epochs=TUNE_EPOCHS)
        metrics = evaluate(model, target_test)
        update_metrics(setup_name, model_name, "four_weeks_tl", metrics['MAE'], metrics['MSE'])

        # Fine-tune on full target train set (full_tl)
        full_tl_model = fine_tune_model(full_tl_model, target_train, epochs=TUNE_EPOCHS)
        metrics = evaluate(full_tl_model, target_test)
        update_metrics(setup_name, model_name, "full_tl", metrics['MAE'], metrics['MSE'])

        # Save after every dataset combination
        results_df.to_csv(metrics_output_path)

def train_baselines(setup_name, tl_data):
    target_fine_tuning = tl_data["target_fine_tuning"]
    target_test = tl_data["target_test"]
    target_train = tl_data["target_train"]
    target_validation = tl_data["target_validation"]

    for model_name, model_func in models.items():
        if is_metric_filled(setup_name, model_name, "four_weeks_training"):
            print(f"Skipping {model_name} for {setup_name} with as metrics are already filled.")
            continue

        # Train on full target train set (full_training)
        model = model_func(target_train, target_validation, epochs=TRAIN_EPOCHS)
        metrics = evaluate(model, target_test)
        update_metrics(setup_name, model_name, "full_training", metrics['MAE'], metrics['MSE'])

        # Train on short target train set (four_weeks_training)
        model = model_func(target_fine_tuning, target_validation, epochs=TRAIN_EPOCHS)
        metrics = evaluate(model, target_test)
        update_metrics(setup_name, model_name, "four_weeks_training", metrics['MAE'], metrics['MSE'])
        
        results_df.to_csv(metrics_output_path)

# Execute for each setup and fine-tuning scenario
for setup_name, (source_data, target_data) in tl_setups.items():
    print(setup_name)
    tl_data = process_tl_data(source_data, target_data)
    run_model_tl(setup_name, tl_data)
    train_baselines(setup_name, tl_data)


# split source/target df into 2 index columns
results_df.to_csv(metrics_output_path)

# Drop rows with NaN values and save the final results
results_df



ELD_to_Bavaria
Skipping NHiTS for ELD_to_Bavaria with full fine-tuning as metrics are already filled.
Skipping NBEATS for ELD_to_Bavaria with full fine-tuning as metrics are already filled.
Skipping Transformer for ELD_to_Bavaria with full fine-tuning as metrics are already filled.
Skipping TSMixer for ELD_to_Bavaria with full fine-tuning as metrics are already filled.
Skipping NHiTS for ELD_to_Bavaria with as metrics are already filled.
Skipping NBEATS for ELD_to_Bavaria with as metrics are already filled.
Skipping Transformer for ELD_to_Bavaria with as metrics are already filled.
Skipping TSMixer for ELD_to_Bavaria with as metrics are already filled.
ELD_to_GP2


/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/torch/random.py:107: UserWarning: CUDA reports that you have 3 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of GPUs.  If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using.  For example, if you are using CPU only, set CUDA_VISIBLE_DEVICES= or devices=[]; if you are using GPU 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` c

Predicting DataLoader 0: 100%|██████████| 18/18 [00:01<00:00, 17.06it/s]


/tmp/ipykernel_15329/3755901336.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
/tmp/ipykernel_15329/3755901336.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Metrics updated for NHiTS in ELD_to_GP2 with zero-shot fine-tuning: MAE = 0.6177704326631382, MSE = 0.9489661906820329


...

Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 31.84it/s]


/tmp/ipykernel_15329/3755901336.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
/tmp/ipykernel_15329/3755901336.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
`Trainer.fit` stopped: `max_epochs=15` reached.
GPU available: True

Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 32.17it/s]
Skipping NBEATS for GP2_to_ELD with as metrics are already filled.
Skipping Transformer for GP2_to_ELD with as metrics are already filled.
Skipping TSMixer for GP2_to_ELD with as metrics are already filled.


/tmp/ipykernel_15329/3755901336.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
/tmp/ipykernel_15329/3755901336.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse


NHiTS        NBEATS  \
Setup          Learning_scenario   Metric                             
ELD_to_Bavaria Zero-Shot           MSE       0.001162  1.088641e-03   
                                   MAE       0.024192  2.262520e-02   
               four_weeks_tl       MSE       0.000353  5.003704e-04   
                                   MAE       0.010298  1.153545e-02   
               full_tl             MSE       0.000305  3.152571e-04   
                                   MAE       0.009601  9.828738e-03   
               full_training       MSE       0.000299  2.923995e-04   
                                   MAE       0.009047  9.455105e-03   
               four_weeks_training MSE       0.000368  5.265365e-04   
                                   MAE       0.010557  1.164277e-02   
ELD_to_GP2     Zero-Shot           MSE       0.948966  9.630598e+15   
                                   MAE       0.617770  3.994391e+07   
               four_weeks_tl       MSE       0.819147  9.964590e+24   
                                   MAE       0.541397  2.019681e+12   
               full_tl             MSE       1.270368  9.505193e+15   
                                   MAE       0.713655  5.947970e+07   
               full_training       MSE       0.659480  4.650204e+13   
                                   MAE       0.464154  3.837060e+06   
               four_weeks_training MSE       0.824351  1.669291e+14   
                                   MAE       0.533056  7.476788e+06   
Bavaria_to_ELD Zero-Shot           MSE      10.242845  5.666167e+20   
                                   MAE       2.700197  1.062929e+10   
               four_weeks_tl       MSE       0.427707  1.846429e+16   
                                   MAE       0.474106  7.225413e+07   
               full_tl             MSE       0.286422  2.197332e+06   
                                   MAE       0.362894  1.061997e+03   
               full_training       MSE       0.279225  2.779544e+06   
                                   MAE       0.360106  1.164405e+03   
               four_weeks_training MSE       0.380317  2.762714e+09   
                                   MAE       0.431907  3.819075e+04   
Bavaria_to_GP2 Zero-Shot           MSE     466.380083  1.561072e+21   
                                   MAE      11.542151  1.603994e+10   
               four_weeks_tl       MSE       1.016150  9.454969e+19   
                                   MAE       0.656820  3.837765e+09   
               full_tl             MSE       0.658909  1.565711e+13   
                                   MAE       0.458732  2.279997e+06   
               full_training       MSE       0.684025  6.799420e+14   
                                   MAE       0.479501  1.416744e+07   
               four_weeks_training MSE       0.824021  5.312447e+17   
                                   MAE       0.529945  4.333730e+08   
GP2_to_Bavaria Zero-Shot           MSE       0.012210  8.759797e-03   
                                   MAE       0.069767  5.770039e-02   
               four_weeks_tl       MSE       0.000589  1.022711e-03   
                                   MAE       0.014388  2.032992e-02   
               full_tl             MSE       0.000293  3.025456e-04   
                                   MAE       0.009276  9.730158e-03   
               full_training       MSE       0.000301  3.206283e-04   
                                   MAE       0.009136  9.783117e-03   
               four_weeks_training MSE       0.000377  5.235213e-04   
                                   MAE       0.010759  1.157042e-02   
GP2_to_ELD     Zero-Shot           MSE       0.784147  2.328977e+04   
                                   MAE       0.710149  8.976374e+01   
               four_weeks_tl       MSE       0.401205  3.652951e+14   
                                   MAE       0.450909  1.352897e+07   
               full_tl             MSE       0.288997  7.574842e+06   
 